In [1]:
import os
import cv2
from ultralytics import YOLO

def load_model(model_path):
    return YOLO(model_path)

def detect_height_bottle(image_path, model):
    results = model(image_path)
    if results and results[0].boxes:
        box = results[0].boxes.xyxy[0].cpu().numpy()
        height_pixels = box[3] - box[1]
        estimated_real_height = 24  # Assuming standard Coca-Cola bottle height in cm
        return height_pixels, estimated_real_height
    return None, None

def detect_objects(model, image_path):
    results = model(image_path)
    return results, cv2.imread(image_path)

def get_floor_coordinates(results):
    if results and results[0].boxes:
        box = results[0].boxes.xyxy[0].cpu().numpy()
        return box[1], box  # Return y-top coordinate of floor
    return None, None

def get_propeller_coordinates(results):
    propeller_y_bottoms = []
    propeller_boxes = []
    if results and results[0].boxes:
        for box in results[0].boxes.xyxy:
            box_np = box.cpu().numpy()
            propeller_y_bottoms.append(box_np[3])  # Bottom y-coordinate
            propeller_boxes.append(box_np)
    return propeller_y_bottoms, propeller_boxes

def calculate_height(propeller_y, floor_y, calibration_obj):
    estimated_real_height, height_pixels = calibration_obj
    pixel_to_cm_ratio = estimated_real_height / height_pixels
    return (floor_y - propeller_y) * pixel_to_cm_ratio

def draw_detections(image, propeller_boxes, floor_box, calibration_obj):
    for box in propeller_boxes:
        prop_x = int((box[0] + box[2]) / 2)
        floor_x = int((floor_box[0] + floor_box[2]) / 2)
        cv2.line(image, (prop_x, int(box[3])), (floor_x, int(floor_box[1])), (0, 255, 0), 2)
        real_height = calculate_height(box[3], floor_box[1], calibration_obj)
        cv2.putText(image, f"{real_height:.2f} cm", (prop_x, int((box[3] + floor_box[1]) / 2)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    return image


In [2]:
image_folder = "Input_test_folder(3m approx)"
output_folder = "Output_test_folder_propeller"
model_floor_path = "best (5).pt"
model_propeller_path = "best (9).pt"
coke_model_path = "best (8).pt"

os.makedirs(output_folder, exist_ok=True)
coke_model = load_model(coke_model_path)
floor_model = load_model(model_floor_path)
propeller_model = load_model(model_propeller_path)

for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
    if height_pixels is None:
        print(f"Skipping {image_name}: No Coca-Cola bottle detected.")
        continue
    
    calibration_obj = (estimated_real_height, height_pixels)
    floor_results, image = detect_objects(floor_model, image_path)
    floor_y_top, floor_box = get_floor_coordinates(floor_results)
    propeller_results, _ = detect_objects(propeller_model, image_path)
    propeller_y_bottoms, propeller_boxes = get_propeller_coordinates(propeller_results)
    
    if not propeller_boxes or floor_y_top is None:
        print(f"No propellers or floor detected in {image_path}")
        continue
    
    print(f"\nImage: {image_path}")
    for i, propeller_y_bottom in enumerate(propeller_y_bottoms):
        real_height = calculate_height(propeller_y_bottom, floor_y_top, calibration_obj)
        print(f"  Propeller {i+1}: {real_height:.2f} cm from floor")
    
    output_image = draw_detections(image, propeller_boxes, floor_box, calibration_obj)
    output_path = os.path.join(output_folder, image_name)
    cv2.imwrite(output_path, output_image)
    print(f"Output saved at: {output_path}")


image 1/1 /home/priyansh/Philipee OBADIA  Project/Input_test_folder(3m approx)/image44.jpeg: 800x608 1 750ml coke, 364.8ms
Speed: 4.4ms preprocess, 364.8ms inference, 0.8ms postprocess per image at shape (1, 3, 800, 608)

image 1/1 /home/priyansh/Philipee OBADIA  Project/Input_test_folder(3m approx)/image44.jpeg: 640x480 1 ceiling, 2 floors, 4 wallss, 338.9ms
Speed: 1.9ms preprocess, 338.9ms inference, 9.4ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/priyansh/Philipee OBADIA  Project/Input_test_folder(3m approx)/image44.jpeg: 800x608 (no detections), 342.0ms
Speed: 3.6ms preprocess, 342.0ms inference, 0.4ms postprocess per image at shape (1, 3, 800, 608)
No propellers or floor detected in Input_test_folder(3m approx)/image44.jpeg

image 1/1 /home/priyansh/Philipee OBADIA  Project/Input_test_folder(3m approx)/image47.jpeg: 800x608 1 750ml coke, 377.7ms
Speed: 5.8ms preprocess, 377.7ms inference, 0.6ms postprocess per image at shape (1, 3, 800, 608)

image 1/1 /hom